In [1]:
import pickle
import json
import matplotlib.pyplot as plt
import re
import time
import requests
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from itertools import combinations
from nltk.tokenize import RegexpTokenizer
from collections import OrderedDict

In [2]:
with open('../finalData//finalDisease.json') as json_file:
    result_dict = json.load(json_file)
print("total diseases:",len(result_dict.keys()))

total diseases: 494


# symptoms cleaning

## Removing diseases with same symptoms

In [3]:
recurring_list=list()
for disease1,data1 in result_dict.items():
    for disease2,data2 in result_dict.items():
        if disease1!=disease2:
            if data1['symptoms']==data2['symptoms']:
                print(disease1,"-->",disease2)
                recurring_list.append(disease1)
for i in range(1,len(recurring_list),2):
    del result_dict[recurring_list[i]]
print("\ntotal diseases after deleting :",len(result_dict.keys()))


total diseases after deleting : 494


## removing symptom key if data is not present in the key
example: ut people may experience

In [4]:
result_dict_temp=result_dict.copy()
count=0
del_keys=list()
for disease,data in result_dict.items():
    temp_key_del=list()
    for key in data['symptoms']:
        if len(data['symptoms'][key])==0:
            count+=1
            temp_key_del.append(key)
    del_keys.extend(temp_key_del)
    for key in temp_key_del:
        del data['symptoms'][key]
print("deleted",count,"keys")
result_dict=result_dict_temp.copy()
print(set(del_keys))

deleted 75 keys
{'ut people may experience'}


## Keys in data

In [5]:
symptoms_list=list()
for disease,data in result_dict.items():
    symptoms_list+=list(data['symptoms'].keys())
symptoms_list=list(set(symptoms_list))
print(sorted(symptoms_list))

['Abdominal', 'Also common', 'Anal', 'Arm', 'Behavioural', 'Breast', 'Chest', 'Cognitive', 'Common symptoms', 'Cough', 'Developmental', 'Ears', 'Eyes', 'Facial', 'Foot', 'Gastrointestinal', 'Groin', 'Hair', 'Hand', 'Head', 'Headache', 'Heart', 'Joints', 'Limbs', 'Menstrual', 'Mood', 'Mouth', 'Muscular', 'Nails', 'Nasal', 'Neck', 'Pain areas', 'Pain circumstances', 'Pain types', 'Psychological', 'Respiratory', 'Sensory', 'Sexual', 'Skin', 'Sleep', 'Speech', 'Throat', 'Tremor', 'Urinary', 'Visual', 'Weight', 'Whole body']


In [6]:
lst=[]
for ty in symptoms_list:
    count=0
    for disease,data in result_dict.items():
            if ty in data['symptoms'].keys():
                count+=1
                
    lst.append([ty,count])
lst.sort(key = lambda x: x[1],reverse=True)
total_sym=0
for disease in lst:
    if disease[1]:
        print(disease[0],disease[1])
        total_sym+=disease[1]
# print("total symptoms:",total_sym)

Also common 419
Pain areas 227
Whole body 178
Gastrointestinal 88
Skin 69
Common symptoms 51
Pain circumstances 47
Muscular 45
Mood 39
Behavioural 35
Respiratory 32
Visual 28
Cognitive 28
Pain types 27
Sensory 26
Eyes 24
Psychological 20
Heart 20
Developmental 19
Mouth 18
Nasal 17
Cough 17
Urinary 17
Sleep 15
Speech 15
Groin 11
Menstrual 11
Weight 9
Joints 8
Throat 7
Abdominal 6
Facial 5
Sexual 5
Hair 4
Headache 4
Anal 4
Breast 4
Hand 3
Ears 3
Neck 3
Tremor 2
Arm 1
Chest 1
Head 1
Nails 1
Foot 1
Limbs 1


In [7]:
updated_keys=['Also common','Whole body','Gastrointestinal','Skin',
              'Common symptoms','Pain areas','Respiratory','Muscular',
              'Mood','Behavioural','Visual','Eyes','Psychological','Heart',
              'Mouth','Groin','Menstrual','Weight']

# finding the most common occurances in the updated keys
#### to find the most common occurance so that if we encounte a case where the key lenght goes beyond length of 9 then we can add consider this list to add

In [8]:
# all_symptoms=list()
def clean(symptoms,typeof):
    if typeof=="Pain areas":
#         print(symptoms)
        symptoms=symptoms[7:]
    symptoms=symptoms.replace('-',' ')
    symptoms = re.sub(r'\bor\b', ',', symptoms)
    symptoms = symptoms.replace(", ,", ",")
    symptoms= symptoms.strip().split(",")
    if typeof=="Pain areas":
        for symp in range(0,len(symptoms)):
            symptoms[symp]=symptoms[symp].strip()
            symptoms[symp]+=" pain"
    else:
        for symp in range(0,len(symptoms)):
            symptoms[symp]=symptoms[symp].strip()
    #all_symptoms.extend(symptoms)
    return symptoms
#     symptoms=symptoms.replace('-',' ')
#     symptoms = re.sub(r'\bor\b', ',', symptoms)
#     symptoms = symptoms.replace(", ,", ",")
#     symptoms= symptoms.strip().split(",")
#     #removing spaces for each symptom
#     for symp in range(0,len(symptoms)):
#         symptoms[symp]=symptoms[symp].strip()
#     #all_symptoms.extend(symptoms)
#     return symptoms

In [9]:
# common_dict={"Also common":[],'Whole body':[],'Gastrointestinal':[],'Skin':[],'Common symptoms':[]}
common_dict=dict()
for key in updated_keys:
    common_dict[key]=list()
for disease,data in result_dict.items():
    for symptom_type in data['symptoms'].keys():
        if symptom_type in updated_keys:
            common_dict[symptom_type].extend(clean(data['symptoms'][symptom_type],symptom_type))
            #common_dict[symptom_type]+=data['symtpoms'][symptom_type]
print(common_dict['Also common'][:40])

['bruising', 'limping', 'pain when standing', 'nausea', 'pulsating abdominal mass', 'difficulty swallowing', 'difficulty belching', 'weight loss', 'bitter taste', 'discomfort in upper abdomen', 'dry cough', 'pimple', 'blackhead', 'chest pressure', 'headache', 'shortness of breath', 'sleeping difficulty', 'sore throat', 'bleeding gums', 'easy bruising', 'infection', 'nosebleed', 'pallor', 'red spots on skin', 'shortness of breath', 'swollen lymph nodes', 'weight loss', 'bleeding', 'bruising', 'infection', 'pallor', 'red spots on skin', 'shortness of breath', 'absence of menstruation', 'craving salty foods', 'darkening of the skin', 'excess urination', 'muscle weakness', 'reduced sex drive', 'weight loss']


In [10]:
for typ in common_dict.keys():
    print(typ,"-->",len(common_dict[typ]))

Also common --> 1703
Whole body --> 589
Gastrointestinal --> 294
Skin --> 257
Common symptoms --> 165
Pain areas --> 431
Respiratory --> 96
Muscular --> 169
Mood --> 169
Behavioural --> 210
Visual --> 93
Eyes --> 80
Psychological --> 67
Heart --> 50
Mouth --> 51
Groin --> 39
Menstrual --> 35
Weight --> 19


### sorting common dict according to the occurance of disease

In [11]:
from collections import defaultdict
def leaders(xs):
    counts = defaultdict(int)
    for x in xs:
        counts[x] += 1
    return sorted(counts.items(), reverse=True, key=lambda tup: tup[1])
for typ in common_dict.keys():
    res=leaders(common_dict[typ])
#     print(res)
    temp=list()
    for item in res:
        temp.append(item[0])
    common_dict[typ]=temp

In [12]:
common_dict["Also common"][:10]

['headache',
 'weight loss',
 'itching',
 'swelling',
 'shortness of breath',
 'depression',
 'swollen lymph nodes',
 'fatigue',
 'tenderness',
 'irritability']

in the above **headache** is the most common occuring symptom among all so these are sorte in that way

# Building disease symptom dataset

In [13]:
all_symptoms=list()
def modify(symptoms,typeof):
    if typeof=="Pain areas":
#         print(symptoms)
        symptoms=symptoms[7:]
    symptoms=symptoms.replace('-',' ')
    symptoms = re.sub(r'\bor\b', ',', symptoms)
    symptoms = symptoms.replace(", ,", ",")
    symptoms= symptoms.strip().split(",")
    symptoms_copy=list()
    if typeof=="Pain areas":
        for symp in range(0,len(symptoms)):
            symptoms[symp]=symptoms[symp].strip()
            symptoms[symp]+=" pain"
            if len(symptoms[symp].split())>7:
                symptoms_copy.append(symptoms[symp])
                print(symptoms[symp])
    else:
        
        for symp in range(0,len(symptoms)):
            symptoms[symp]=symptoms[symp].strip()
            if len(symptoms[symp].split())>7:
                symptoms_copy.append(symptoms[symp])
                print(symptoms[symp])
    for dis in symptoms_copy:
        symptoms.remove(dis)
    all_symptoms.extend(symptoms)
    return symptoms

In [14]:
#remove duplicates from a list while preserving order
def customSort(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [15]:
diseases_symptoms_cleaned=OrderedDict()
symptom_length=9
for disease,data in result_dict.items():
    symptoms=list()
    for key in updated_keys:
        if((key in data['symptoms'].keys()) and (len(symptoms)<=symptom_length)):
            #modify
            temp_symp=modify(data['symptoms'][key],key)
            #sort temp_symp in the common dict order
            value_index=[common_dict[key].index(x) for x in temp_symp]
            sorted_temp_symp=[temp_symp for _, temp_symp in sorted(zip(value_index, temp_symp))]
            if len(symptoms)+len(temp_symp)>symptom_length:
                buffer=symptom_length-len(symptoms)
                symptoms.extend(sorted_temp_symp[:buffer])
#                 print(temp_symp)
#                 print(value_index)
#                 print(sorted_temp_symp)
            else:
                symptoms.extend(sorted_temp_symp)
    symptoms=customSort(symptoms)
    diseases_symptoms_cleaned[disease]=symptoms

intense interest in a limited number of things
web of swollen blood vessels in the skin
both eyes not focused on same object at same time
thighs and buttocks that worsens with standing pain
excess skin on the back of the neck
thickening of the skin of the palms and soles
web of swollen blood vessels in the skin
web of swollen blood vessels in the skin
web of swollen blood vessels in the skin
a rim of discolouration around the front of the eye (fleischer ring)
thighs and buttocks that worsens with standing pain
thighs and buttocks that worsens with standing pain
thighs and buttocks that worsens with standing pain


In [16]:
count=0
lst=[]
number_count=dict()
disease_remove_list=list()
for disease,symptom in diseases_symptoms_cleaned.items():
    lent=len(symptom)
    if lent==0:
        disease_remove_list.append(disease)
        continue
    if lent in number_count.keys():
        number_count[lent]+=1
    else:
        number_count[lent]=1
    lst.append(lent)
for dis in disease_remove_list:
    del diseases_symptoms_cleaned[dis]

print("number count dictonary")
for i in sorted(number_count) :
    print ((i, number_count[i]), end =" ")
print("\ntotal number of diseases:",len(diseases_symptoms_cleaned))
print("total number of unique symptoms:",len(set(all_symptoms)))


number count dictonary
(1, 12) (2, 28) (3, 30) (4, 29) (5, 43) (6, 41) (7, 50) (8, 46) (9, 212) 
total number of diseases: 491
total number of unique symptoms: 1000


In [17]:
total=0
for i in sorted(number_count) :
    sumi=(2**i)*number_count[i]
    total+=sumi
    print ((i, sumi), end =" ")
print("\ntotal rows",total)

(1, 24) (2, 112) (3, 240) (4, 464) (5, 1376) (6, 2624) (7, 6400) (8, 11776) (9, 108544) 
total rows 131560


In [18]:
all_symptoms=list(set(all_symptoms))
all_symptoms.sort()
len(all_symptoms)

1000

In [19]:
# all_symptoms_temp=list()
# for disease,symptom in diseases_symptoms_cleaned.items():
# #     print(symptom)
#     all_symptoms_temp.extend(symptom)
# len(set(all_symptoms_temp))
# all_symptoms=list(set(all_symptoms_temp)).copy()
# all_symptoms.sort()

# finding meaning for words

### https://www.thesaurus.com/

In [20]:
def write(sy_sy):
    with open('../finalData/pickle/sym_syn.pickle', 'wb') as handle:
        pickle.dump(sy_sy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open('../finalData/pickle/sym_syn.pickle', 'rb') as handle:
    sym_syn = pickle.load(handle)
len(sym_syn)

1140

In [22]:
## if there are any symptoms that are not in lexical dictionary this will print them
count=0
for s in sorted(all_symptoms):
    s=s.replace('-',' ')
    s=s.replace("'",'')
    s=s.replace('(','')
    s=s.replace(')','')
    if s not in sym_syn.keys():
#         print(s)
        count+=1
print(count,"missing symptoms")

0 missing symptoms


In [23]:
total_symptoms=all_symptoms.copy()
total_symptoms = sorted(total_symptoms, key=len, reverse=True)
print("total symptoms:",len(total_symptoms))

total symptoms: 1000


In [24]:
target=open("../finalData/log.txt","a")
def synonyms(term):
    synonyms = []
    try:
        response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    except:
        time.sleep(60)
        response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.content,  "html.parser")
    try:
        container=soup.find('section', {'class': 'MainContentContainer'}) 
        # row=container.find('div',{'class':'css-191l5o0-ClassicContentCard'})
        row=container.find('div',{'class':'css-i3pbo e1i572590'})
        row = row.find_all('li')
        for x in row:
            synonyms.append(x.get_text())
    except:
        target.write(term+"\n")
        target.flush()
#         print(term,None)
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms) 

In [25]:
start_time = time.time()
# stores each symptom's synonyms as a list of words
total_symptoms=all_symptoms.copy()
#sym_syn = dict()
count=0
# for s in total_symptoms[870]:
for s in total_symptoms:
    s=s.replace('-',' ')
    s=s.replace("'",'')
    s=s.replace('(','')
    s=s.replace(')','')
    if s in sym_syn.keys():
        continue
    target.write("------------------------->"+s+"\n")
    target.flush()
    count+=1
    symp=s.split()
    str_sym=set()
    for comb in range(1, len(symp)+1):
        for subset in combinations(symp, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset)
    str_sym.add(s)
    str_sym = ' '.join(str_sym).replace('_',' ').lower()
    str_sym = list(set(str_sym.split()))
    str_sym.sort()
    sym_syn[s] = str_sym
    target.write(str(str_sym)+"\n")
    target.write(30*"*"+"\n")
    target.flush()
    ## store in pickle file for every 50 diseases
    if count== 30:
        write(sym_syn)
        count=0
write(sym_syn)
print("Done!",count)
print("--- %s seconds ---" % (time.time() - start_time))

Done! 0
--- 0.020943641662597656 seconds ---


### Removing matching symptoms with more than 80%

In [49]:
start_time = time.time()
def modify(s):
    s=s.replace('-',' ')
    s=s.replace("'",'')
    s=s.replace('(','')
    s=s.replace(')','')
    return s
symptom_match=dict()
new_symptoms = set()
for i,symi in enumerate(total_symptoms):
    symi=modify(symi)
    for j in range(i+1,len(total_symptoms)):
        symj=total_symptoms[j]
        symj=modify(symj)
        syn_symi = set(sym_syn[symi])
        syn_symj = set(sym_syn[symj])
        jaccard = len(syn_symi.intersection(syn_symj))/len(syn_symi.union(syn_symj))
        if jaccard>0.80:
            if "pain" in symi:
                if jaccard<0.90:
                    continue
            print(symi,"->",symj)
            # Store similar symptoms in dictionary, replace symj with symi, so  
            # symptom_match[symj] = symi
            if symi in symptom_match.keys():
                symptom_match[symj] =  symptom_match[symi]
            else:
                symptom_match[symj] = symi
new_symptoms = set(total_symptoms).difference(set(symptom_match.keys()))
print("--- %s seconds ---" % (time.time() - start_time))

abdomen pain -> mid abdomen pain
atlantoaxial instability -> instability
blackhead -> follicular papules
calf pain -> rectum pain
coeliac disease -> peripheral artery disease
compulsive behaviour -> disorganised behaviour
compulsive behaviour -> ritualistic behaviour
discharge -> increased vaginal discharge
discharge -> vaginal discharge
disorganised behaviour -> ritualistic behaviour
enlarged clitoris -> enlarged thyroid
eyelid pain -> rectum pain
eyelid redness -> redness
facial muscle weakness -> muscle weakness
forearm pain -> rectum pain
frequent infections -> frequent respiratory infections
heavy -> heavy menstruation
infection -> opportunistic infection
infection -> recurrent infection
leg pain -> legs pain
light headedness -> light sensitivity
lump -> lumps
mouth ulcer -> oral thrush mouth thrush
noisy breathing -> rapid breathing
noisy breathing -> shallow breathing
palpitations -> shakiness
personality change -> personality changes
premature hair whitening -> pubic hair
purpl

In [50]:
len(symptom_match)

32

In [51]:
symp=list()
for disease,symptoms in diseases_symptoms_cleaned.items():
    symp+=symptoms
new_symptoms=list(set(symp))
new_symptoms.sort()

print("final disease list:",len(diseases_symptoms_cleaned))
print("final symptoms:",len(new_symptoms))

final disease list: 491
final symptoms: 820


# creating Analyze files

In [54]:
f = open("../finalData/Analyze/disease_symptoms_cleaned.txt", "w")
for dis in sorted(diseases_symptoms_cleaned.keys()):
    f.write(dis+" --> "+str(diseases_symptoms_cleaned[dis])+"\n")
f.close()
f = open("../finalData/Analyze/1.diseases.txt", "w")
for dis in sorted(diseases_symptoms_cleaned.keys()):
    f.write(dis+"\n")
f.close()
f = open("../finalData/Analyze/2.all_symptoms.txt", "w")
for symptom in all_symptoms:
    f.write(symptom+"\n")
f.close()

In [56]:
with open('../finalData/pickle/diseases_symptoms_cleaned.pickle', 'wb') as handle:
    pickle.dump(diseases_symptoms_cleaned, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../finalData/pickle/new_symptoms.pickle', 'wb') as handle:
    pickle.dump(new_symptoms, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../finalData/pickle/symptom_match.pickle', 'wb') as handle:
    pickle.dump(symptom_match, handle, protocol=pickle.HIGHEST_PROTOCOL)